# 线性回归

- 回归(regression)：建模自变量与因变量之间关系，表示输入与输出关系。通常执行预测(prediction)任务。如预测价格、预测需求量

## 线性回归基本元素
- training data, or training set
- 每行数据(sample/data point/data instance)
- 预测目标（如预测房价）(label/target)
- 预测所依据的自变量（如面积、房龄等）(feature/covariate)
- 偏置量是当所有参数为0时，预测值应该是多少(bias/offset/intercept)，若没有会限制模型表达能力
- 权重为自变量对目标变量的影响程度(weight)

### 线性模型
- target为房屋价格
- feature为房龄和面积
- weight为$\omega_{area}$和$\omega_{age}$
- bias为$b$
  $$ price=\omega_{area} \cdot area+\omega_{age} \cdot age+b$$
- 这个模型是输入特征的一个仿射变换(affine transformation)，其特点是通过weight和对feature进行线性变换(linear transformation)，并通过bias来进行平移(translation)
  
#### 也就是说
在机器学习中，预测结果$\hat{y}$表示为：
$$ \hat{y}=\omega_1 x_1+\omega_2 x_2+\cdots+\omega_n x_n+b=\omega^\top x+b $$

当$\mathbf{X}$ 每一行是一个sample，每一列是一个feature时，即 $\mathbf{X}=\begin{bmatrix}x_1^T\\x_2^T\\\vdots\\x_n^\top\end{bmatrix}$时，
$$\hat{y}=\mathbf{X}\mathbf{\omega}+b$$

- 这个过程中求和使用broadcast  
- 当给定training data feature $\mathbf{X}$, label $y$ 时，线性回归的target是找到一组weight $\mathbf{\omega}$和bias $b$，这组数据能使新sample的预测结果尽量接近真实值
- 在寻找model parameters($\omega$和$b$)之前，我们还需要
  1. 模型质量的度量方式
  2. 能更新模型的方法，以达到更高模型质量

### 损失函数(loss function)
- 量化实际值与预测值的差距
- 通常选用正数作为损失，越小越好
- 回归问题中最常用的时平方误差函数
  $$ l^{(i)}(\omega,b)=\frac{1}{2}(y^{(i)}-\hat{y}^{(i)})^2 $$
- 以n个样本的损失均值作为损失函数
  $$ L(\omega,b)=\frac{1}{n}\sum_{i=1}^n l^{(i)}(\omega,b) = \frac{1}{n}\sum_{i=1}^n (y^{(i)}-\omega^\top x^{(i)}-b)^2$$
- 我们的目的是寻找一组parameters $\omega^*,b^*$，使得损失函数最小化
  $$ \omega^*,b^*=\arg\min_{\omega,b} L(\omega,b) $$

### 解析解
- 线性回归的解可以用一个公式表达出来，称为解析解(analytical solution)
- 将$b$ 合并到$\omega$中，即$\hat{y}$的增广（augment matrix），具体过程为：
  1. 合并
  $$
  \mathbf{X}=\begin{bmatrix}
  \mathbf{X}&|&1
  \end{bmatrix}
  =\begin{bmatrix}
  X_1&|&1\\
  X_2&|&1\\
  \vdots&|&\vdots\\
  X_n&|&1
  \end{bmatrix}, 
  \omega=\begin{bmatrix}
  \omega_1\\
  \omega_2\\
  \vdots\\
  \omega_n\\
  \hline\\
  b
  \end{bmatrix}
  \Rightarrow
  \hat{y}=\mathbf{X}\cdot \omega=\begin{bmatrix}
  X_1 \omega\\
  X_2 \omega\\
  \vdots\\
  X_n \omega
  \end{bmatrix}\\
  $$
  2. 定义损失函数
  $$
  \Rightarrow
  L=\|\hat{y}-y\|^2=(\mathbf{X}\omega-y)^\top (\mathbf{X}\omega-y)=\omega^\top \mathbf{X}^\top \mathbf{X}\omega-\omega^\top \mathbf{X}^\top y-y^\top \mathbf{X}\omega+y^\top y\\
  
  $$
  3. 求导：根据ch2:4.calculus.ipynb中梯度内容可得
   $$
   \begin{aligned}
   \Rightarrow
  \frac{\partial L}{\partial \omega}&=\frac{\partial (\omega^\top \mathbf{X}^\top \mathbf{X}\omega)}{\partial \omega}-\frac{\partial (\omega^\top \mathbf{X}^\top y)}{\partial \omega}-\frac{\partial (y^\top \mathbf{X}\omega)}{\partial \omega}-\frac{\partial (y^\top y)}{\partial \omega}\\
  &=2\mathbf{X}^\top \mathbf{X}\omega-\mathbf{X}^\top y-\mathbf{X}^\top y=2\mathbf{X}^\top \mathbf{X}\omega-2\mathbf{X}^\top y
  \end{aligned}
  $$

  4. 求解loss function最小值：
  $$
  \frac{\partial L}{\partial \omega}=0\Rightarrow 2\mathbf{X}^\top \mathbf{X}\omega-2\mathbf{X}^\top y=0
  \Rightarrow \mathbf{X}^\top \mathbf{X}\omega= \mathbf{X}^\top y$$
    - 若$\mathbf{X}^\top \mathbf{X}$可逆
  $$\Rightarrow \omega^*=(X^\top X)^{-1}X^Ty$$
- 并不是所有问题都有解析解


### 随机梯度下降
- 即使没有解析解，也能有效训练模型
- 梯度下降(gradient descent)：计算loss function对于model parameters的导数，并更新参数，但可能非常慢，因为需要遍历整个数据集
- 因此在需要更新时随机抽取一批样本，称为小批量随机梯度下降(minibatch stochastic gradient descent)。即在每次迭代中随机抽样固定数量小批量$\beta$，计算小样本梯度，最后将梯度乘以一个预先确定的正数$\eta$，并从当前参数的值中减去。
  $$(\omega,b)\leftarrow(\omega,b)-\frac{\eta}{|\beta|}\sum_{i\in\beta} \partial_{(\omega,b)} l^{(i)}(\omega,b)$$

- 总结
  - loss function：平方损失
  - 算法步骤：
    1. 初始化模型参数的值，如随机初始化
    2. 从数据集中随机抽取小批量样本且在负梯度的方向上更新参数，并不断迭代。
  - 对于平方损失和仿射变换，即为如下形式
    $$
    \begin{aligned}
    \omega &\leftarrow \omega-\frac{\eta}{|\beta|} \sum_{i\in\beta} \partial_\omega l^{(i)}(\omega,b)
    =\omega-\frac{\eta}{|\beta|} \sum_{i\in\beta}x^{(i)}(\omega^\top x^{(i)}+b-y^{(i)}) \\
    b &\leftarrow b-\frac{\eta}{|\beta|} \sum_{i\in\beta}\partial_b l^{(i)}(\omega,b)=b-\frac{\eta}{|\beta|} \sum_{i\in\beta} (\omega^\top x^{(i)}+b-y^{(i)})
    \end{aligned}
    $$
  - 其中$\omega$和$x$都是向量
  - $|\beta|$是小批量样本数(batch size), $\eta$是学习率(learning rate)，通常手动指定。这种手动指定的参数叫超参数(hyperparameter)，通过在验证数据集(validation dataset)上得到的训练迭代结果调整参数(hyperparameter tuning)
  - 训练结束后得到$\hat{\omega},\hat{b}$，但是这些估计值不会使loss function最小化，因为是向最小值缓慢收敛，而不是在有限步数内精确得到最小值
  - linear regression恰好只有一个最小值，但深度神经网络等复杂模型往往有多个最小值，所有挑战为找到一组参数，能在没见过的数据上实现较低损失，这一挑战即为泛化(generalization)

### 用模型进行预测
给定特征估计目标的过程称为预测(prediction)或推断(inference)